<a href="https://colab.research.google.com/github/chetools/CHE4061_Fall2025/blob/main/VLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!wget -N -q https://raw.githubusercontent.com/chetools/chetools/main/tools/che5.ipynb -O che5.ipynb
!pip install importnb

In [8]:
from importnb import Notebook
with Notebook():
    from che5 import Props

import numpy as np
import scipy as sp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_dark"
import jax
import jax.numpy as jnp
jax.config.update("jax_enable_x64", True)

In [3]:
p=Props(['Benzene','Toluene'])

In [9]:
p.Pvap(350)

Array([161727.62817591,  41697.97094244], dtype=float64)

In [5]:
z=np.array([0.4,0.6])

In [10]:
def bubbleP(x, T):
    Pi = x*p.Pvap(T)
    P=np.sum(Pi)
    return P, Pi/P

In [15]:
def dewP(y, T):
    Pisat = p.Pvap(T)
    P=1/np.sum(y/Pisat)
    return P, y*P/Pisat

In [42]:
def bubbleT(x, P):


    T = sp.optimize.root_scalar(lambda T: bubbleP(x, T)[0] - P, x0=300., method='Newton').root

    return T, x*p.Pvap(T)/P

In [54]:
def dewT(y, P):

    def fun_to_zero(v):
        T = v[0]
        x = v[1:]
        return x*p.Pvap(T)-y*P, np.sum(x)-1

    v0 = np.r_[350., y]

    return fun_to_zero(v0)

In [55]:
dewT(z, 101325)

(Array([ 24161.05127037, -35776.21743453], dtype=float64), np.float64(0.0))

In [40]:
bubbleP(z, 350)

(Array(89709.83383583, dtype=float64),
 Array([0.72111438, 0.27888562], dtype=float64))

In [43]:
bubbleT(z, 89709.83383583)


(Array(350., dtype=float64), Array([0.72111438, 0.27888562], dtype=float64))